In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict, RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

import warnings
from IPython import display
import re

In [2]:
train = pd.read_csv('train_regression.csv')
test = pd.read_csv('test_regression.csv')
train.head()

id    host_id  host_since host_location host_response_time  \
0            53872917  366516391  2018-04-09   Chicago, IL     within an hour   
1            39269663   50276775  2014-12-23           NaN     within an hour   
2  556480733030442463  681680651  2020-03-08           NaN     within an hour   
3            52397280   94345291  2015-10-22   Chicago, IL     within an hour   
4  745949202271858960  790998106  2021-04-04   Chicago, IL     within an hour   

  host_response_rate host_acceptance_rate host_is_superhost  \
0                96%                  95%                 f   
1               100%                  97%                 f   
2               100%                 100%                 f   
3               100%                  98%                 f   
4               100%                 100%                 t   

  host_neighbourhood  host_listings_count  ...  review_scores_checkin  \
0  Back of the Yards                   11  ...                   5.00   
1   Clearwater Beach                 1141  ...                    NaN   
2     Lake View East                    1  ...                   5.00   
3           Bucktown                   55  ...                   4.38   
4    Near South Side                   74  ...                   4.83   

  review_scores_communication review_scores_location review_scores_value  \
0                        5.00                   4.28                4.83   
1                         NaN                    NaN                 NaN   
2                        5.00                   4.93                4.80   
3                        4.23                   4.54                3.85   
4                        4.85                   4.74                4.72   

  instant_bookable  calculated_host_listings_count  \
0                f                               9   
1                t                              58   
2                f                               1   
3                t                              55   
4                t                              74   

   calculated_host_listings_count_entire_homes  \
0                                            1   
1                                            0   
2                                            1   
3                                           55   
4                                           74   

  calculated_host_listings_count_private_rooms  \
0                                            8   
1                                           58   
2                                            0   
3                                            0   
4                                            0   

  calculated_host_listings_count_shared_rooms  reviews_per_month  
0                                           0               0.33  
1                                           0                NaN  
2                                           0               2.78  
3                                           0               3.61  
4                                           0               3.24  

[5 rows x 54 columns]

In [15]:
train.dropna()

id    host_id  host_since    host_location  \
0               53872917  366516391  2018-04-09      Chicago, IL   
3               52397280   94345291  2015-10-22      Chicago, IL   
4     745949202271858960  790998106  2021-04-04      Chicago, IL   
6     841653025955157554  570167059  2019-08-12      Chicago, IL   
8     856428289430147957   44110882  2014-10-02      Chicago, IL   
...                  ...        ...         ...              ...   
4993            51606037  341571126  2018-01-30  Springfield, IL   
4995            41561598  469571763  2019-01-06      Chicago, IL   
4996            38350907  981504375  2022-12-08      Chicago, IL   
4997  913493910258754935  284822054  2017-07-25      Chicago, IL   
4999  705815628392534448    1382147  2011-06-11      Chicago, IL   

      host_response_time host_response_rate host_acceptance_rate  \
0         within an hour                96%                  95%   
3         within an hour               100%                  98%   
4         within an hour               100%                 100%   
6     within a few hours               100%                  97%   
8     within a few hours               100%                  93%   
...                  ...                ...                  ...   
4993      within an hour                99%                  97%   
4995  within a few hours                88%                  84%   
4996      within an hour               100%                  89%   
4997      within an hour               100%                  97%   
4999  within a few hours               100%                  87%   

     host_is_superhost host_neighbourhood  host_listings_count  ...  \
0                    f  Back of the Yards                   11  ...   
3                    f           Bucktown                   55  ...   
4                    t    Near South Side                   74  ...   
6                    t          Hyde Park                    3  ...   
8                    f            Douglas                    1  ...   
...                ...                ...                  ...  ...   
4993                 f    Near North Side                  120  ...   
4995                 t        Rogers Park                    2  ...   
4996                 t        Irving Park                    8  ...   
4997                 f         North Park                    4  ...   
4999                 f          Englewood                   22  ...   

      review_scores_checkin review_scores_communication  \
0                      5.00                        5.00   
3                      4.38                        4.23   
4                      4.83                        4.85   
6                      4.69                        4.86   
8                      4.88                        4.94   
...                     ...                         ...   
4993                   4.91                        4.91   
4995                   5.00                        4.99   
4996                   4.75                        4.90   
4997                   4.94                        5.00   
4999                   5.00                        5.00   

     review_scores_location review_scores_value instant_bookable  \
0                      4.28                4.83                f   
3                      4.54                3.85                t   
4                      4.74                4.72                t   
6                      4.43                4.63                f   
8                      4.78                4.91                f   
...                     ...                 ...              ...   
4993                   4.84                4.91                f   
4995                   4.98                4.96                f   
4996                   4.98                4.56                f   
4997                   4.97                4.86                f   
4999                   5.00                5.00                f   

      cal

In [3]:
pd.set_option('display.max_columns', None)

train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage

# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)

train.head()

#All columns in the dataset, except id, host_id, host_since, first_review, and last_review have
#to be tried as predictors. (price is the response, so it cannot be used as a predictor either.)

C:\Users\benja\AppData\Local\Temp\ipykernel_16748\2113949972.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)


id  host_response_rate  host_acceptance_rate  \
0            53872917                96.0                  95.0   
1            39269663               100.0                  97.0   
2  556480733030442463               100.0                 100.0   
3            52397280               100.0                  98.0   
4  745949202271858960               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds   price  minimum_nights  maximum_nights  \
0             1   1.0    30.0              32            1125   
1            12   3.0  1247.0              32             365   
2             6   3.0   214.0               2              45   
3             2   1.0    59.0               2             180   
4             6   3.0   151.0               2             365   

   minimum_minimum_nights  maximum_minimum_nights  minimum_maximum_nights  \
0                      32                      32                    1125   
1                      32                      32                     365   
2                       2                       2                      45   
3                       2                       2                     180   
4                       2                       2                    1125   

   maximum_maximum_nights  minimum_nights_avg_ntm  maximum_nights_avg_ntm  \
0                    1125                    32.0                  1125.0   
1                     365                    32.0                   365.0   
2                      45                     2.0                    45.0   
3                     180                     2.0                   180.0   
4                    1125                     2.0                  1125.0   

   has_availability  availability_30  availability_60  availability_90  \
0                 1               30               60               90   
1                 1               25               49               66   
2                 1                0                0                0   
3                 1               23               53               83   
4                 1                0                0                0   

   availability_365  number_of_reviews  number_of_reviews_ltm  \
0               365                 18                      0   
1               135                  0                      0   
2                 0                 14                     14   
3               355                 13                     13   
4                 0                 64                     42   

   number_of_reviews_l30d  review_scores_rating  review_scores_accuracy  \
0                       0              4.940000                 5.00000   
1                       0              4.759538                 4.79342   
2                       1              4.870000                 5.00000   
3                       3              4.080000                 4.08000   
4                       3              4.800000                 4.74000   

   review_scores_cleanliness  review_scores_checkin  \
0                   4.890000               5.000000   
1                   4.747335  

In [4]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [5]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

In [6]:
base_model = DecisionTreeRegressor(random_state=1, max_depth=5) # hyper param 1 here

model = AdaBoostRegressor(estimator=base_model, random_state=1, n_estimators=250, learning_rate=0.1) # hyper param 2 and 3 here

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [8]:
# Narrowing features with lasso regression

x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

sc = StandardScaler()

sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_test_scaled = sc.transform(x_test)

lasso_model = Lasso(alpha=0.005, max_iter=400000)

x_train_subset = x_train_scaled[:500, :]

lasso_model.fit(x_train_subset, y_train[:500])

coefficients = pd.DataFrame({
    'feature': x_train.columns,
    'importance': np.abs(lasso_model.coef_)
})

coefficients.sort_values(by='importance', ascending=False, inplace=True)

coefficients

feature  importance
2                   host_listings_count  331.177902
34       calculated_host_listings_count  327.499061
151        host_neighbourhood_Englewood  124.476203
8                          accommodates   90.040897
256         host_neighbourhood_Woodlawn   85.206021
..                                  ...         ...
101          host_location_Wauconda, IL    0.000000
269  neighbourhood_cleansed_Avalon Park    0.000000
99          host_location_Tulum, Mexico    0.000000
272      neighbourhood_cleansed_Beverly    0.000000
190  host_neighbourhood_Marshall Square    0.000000

[380 rows x 2 columns]

In [9]:
important_features = list(coefficients.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

model = AdaBoostRegressor(estimator=base_model, random_state=1, n_estimators=250, learning_rate=0.1) # hyper param 2 and 3 here

model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [11]:
# coarse grid

base_model = DecisionTreeRegressor(random_state=1) 

model = AdaBoostRegressor(estimator=base_model, random_state=1) 


grid = {
        'estimator__max_depth': range(2,11,2),
        'n_estimators': range(20,150,10),
        'learning_rate': [0.001, 0.01, 0.1, 1, 10]
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 325 candidates, totalling 1625 fits
{'estimator__max_depth': 10, 'learning_rate': 0.01, 'n_estimators': 50}
760.0945240052141


In [13]:
# fine grid

base_model = DecisionTreeRegressor(random_state=1) 

model = AdaBoostRegressor(estimator=base_model, random_state=1) 


grid = {
        'estimator__max_depth': range(10),
        'n_estimators': range(40,60,2),
        'learning_rate': [0.01, 0.02, 0.05, 0.08]
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


C:\Users\benja\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
200 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\benja\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\benja\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\benja\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 169, in fit
    samp

{'estimator__max_depth': 9, 'learning_rate': 0.02, 'n_estimators': 46}
761.5332969984526


In [15]:
# finest grid

base_model = DecisionTreeRegressor(random_state=1) 

model = AdaBoostRegressor(estimator=base_model, random_state=1) 


grid = {
        'estimator__max_depth': [10],
        'n_estimators': range(44,55),
        'learning_rate': [0.01, 0.02, 0.03, 0.04]
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 44 candidates, totalling 220 fits
{'estimator__max_depth': 10, 'learning_rate': 0.01, 'n_estimators': 47}
759.8210409662416


In [17]:
# cv settings, shuffle = true

cv_settings = KFold(n_splits=8, shuffle=True, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 8 folds for each of 44 candidates, totalling 352 fits
{'estimator__max_depth': 10, 'learning_rate': 0.01, 'n_estimators': 48}
1099.698503655609


In [19]:
# cv settings, n_repeats = 5, n_splits =5

cv_settings = RepeatedKFold(n_splits=5, n_repeats=5, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 25 folds for each of 44 candidates, totalling 1100 fits
{'estimator__max_depth': 10, 'learning_rate': 0.01, 'n_estimators': 44}
1112.7352562940403


In [21]:
# Narrow with decision tree importance
importances = np.array(gscv.best_estimator_.feature_importances_)
names = np.array(gscv.feature_names_in_)

important_features = pd.DataFrame({'feature':names, 'importance':importances})

important_features.sort_values(by='importance', inplace=True, ascending=False)



important_features = list(important_features.head(15).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

cv_settings = RepeatedKFold(n_splits=5, n_repeats=5, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 25 folds for each of 44 candidates, totalling 1100 fits
{'estimator__max_depth': 10, 'learning_rate': 0.01, 'n_estimators': 44}
869.7823109728031


In [23]:
# Narrowing features with permutation importance

model = gscv.best_estimator_
model.fit(x_train_reduced, y_train)

perm_importance = permutation_importance(model, x_train_reduced, y_train, n_repeats=10, random_state=1)

feature_names = [col for col in x_train_reduced if col != 'id' and col != 'price']
features_importance = pd.DataFrame({'feature': feature_names, 'importance': perm_importance.importances_mean})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
0                              longitude    1.078453
5        host_neighbourhood_Wrigleyville    0.978456
2                        availability_90    0.883038
1                 review_scores_location    0.784059
3                              bathrooms    0.017871
4                           accommodates    0.008737
8              host_total_listings_count    0.003285
10        calculated_host_listings_count    0.002034
6                 maximum_minimum_nights    0.001660
7                         minimum_nights    0.001236
9                    host_listings_count    0.000956
11                minimum_minimum_nights    0.000511
12  host_neighbourhood_Ukrainian Village    0.000321
13       host_neighbourhood_Pulaski Park    0.000197
14    host_neighbourhood_Near South Side    0.000020

In [25]:
important_features = list(features_importance.head(10).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

model = gscv.best_estimator_

model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [26]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('Adaboost_Regression_submission.csv', index=False)
results.head(30)

id   predicted
0   771986218856585018  164.692308
1   855276028675941785  137.752577
2             48537824  231.396552
3             41867473  150.920000
4             28361473  100.041958
5             51911862   92.974886
6   579514121718775234  108.644518
7             46242944   34.757895
8              5571262   78.221130
9   594937541144766707  137.009852
10  772410051557583562  189.264407
11            13165625   80.830645
12            29191606  144.394286
13            32381296  371.857143
14            50451083  151.082474
15            51565135  109.643123
16            47176352  113.463357
17  924507573112993030   47.089928
18            45653951  198.126761
19            26795754  113.394937
20            42035578   49.653846
21            49742536  113.463357
22            48932073  107.789790
23            15904833  133.555556
24  759540854095218350  107.789790
25            29885798  111.948980
26  655444310450112562  181.095238
27            11527162   77.866782
28            18735326  145.444954
29  606007670793701048  202.720000